# Linear Mixed Models

From this tutorial: https://stats.idre.ucla.edu/other/mult-pkg/introduction-to-linear-mixed-models/

Linear Mixed Models (LMM) are often used to analyse data that are non-indeendent, multilevel, longditudinal or hierarchichal. 


## Maths

### Extending Linear Models
Linear mixed models are an extension of siple linear models to allow both fixed and random effects. They are particularly used in cases where data is not i.i.d, such as data that arises from hierarchical structures. 

When there exists a hierarchy, for example in the case of patients outcomes under different doctors, patient level observations cannot be considered as independent of each other. 



There are multiple ways to deal with such data, the most common way is to aggregate observations. This means averaging the patients from each doctor producing an corrolary independent dataset. Agregates however do not allow us to make full use of our data.

Another option is to analyse the data one unit at a time and build a regressor for each subpopulation. This works however it does not make use of any of the information between doctors.

LMM combine these two alternatives to take into account both the within population data (WP) and the between population data (BP). There can be important lessons we can learn from the differences in WP and BP variance.




###  Including Random Effects

The core of mixed models is that they incorporate fixed and random effects. 

A fixed effect is a parameter that does not vary. For example, we may assume there is some true regression in a population we shall call this $\beta$, and we'll look to find some estimate of it $\hat{\beta}$.  

A random effect is a parameter that is itself a random variable. For example we could say the $\beta$ is distributed as a radnom normal variate with mean $\mu$ and standard deviation $\sigma$

$$\beta \sim \mathcal{N}(\mu, \sigma)$$

This is the same as in linear regression, where we assume the data are random variabes, but the parameters are fixed affects. The data are still random variables, but now the parameters are also random variables at one level, and fixed effects at another level.

### Maths behind LMM

Consider the equation 
$$
\mathbf{y} = \boldsymbol{X\beta} + \boldsymbol{Zu} + \boldsymbol{\varepsilon}
$$

Where $\mathbf{y}$ the outcome is a $N \times 1$ column vector; $\boldsymbol{X}$ is a $N \times p$ matrix of the $p$ predictor variables; $\boldsymbol{\beta}$ is a $p \times 1$ column vector of fixed effect regression coefficients (the $\boldsymbol{\beta}$s); $\boldsymbol{Z}$ is the $N \times qJ$ design matrix for the $q$ random effects and $J$ groups; and $\boldsymbol{\varepsilon}$ is a $n \times 1$ column vector of the residuals, that part of $\mathbf{y}$ that is not explained by the model $\boldsymbol{X\beta} + \boldsymbol{Zu}$

$$
\overbrace{\mathbf{y}}^{\mbox{N x 1}} \quad = \quad
\overbrace{\underbrace{\mathbf{X}}_{\mbox{N x p}} \quad \underbrace{\boldsymbol{\beta}}_{\mbox{p x 1}}}^{\mbox{N x 1}} \quad + \quad
\overbrace{\underbrace{\mathbf{Z}}_{\mbox{N x qJ}} \quad \underbrace{\boldsymbol{u}}_{\mbox{qJ x 1}}}^{\mbox{N x 1}} \quad + \quad\overbrace{\boldsymbol{\varepsilon}}^{\mbox{N x 1}}
$$

### Example

Consider an example from a simulated dataset. Doctors $(J=407)$ indexed by the $j$ subscript each see $n_j$ patients. So our grouping variable is the doctor. Not every doctor sees the same number of patients, randing from just 2 pateints all the way to 40 and averaging around 21. The total number of patients is the sum of the patients seen by each doctor.

$$N = \sum^J_jn_j$$

In our example, $N=8525$ patients were seen by doctors. Our outcome $\textbf{y}$ is a continuous variable, mobility scores. Suppose we had 6 fixed effects, Age, Married, Sex, Red Blood Cell (RBC) count, and White Blood Cell (WBC) count plus a fixed intercept and one random intercept $(q=1)$ for each of the $J=407$ doctors. For simlicity, we are going to consider reandom intercepts. We will let every other effect be fixed for now. The reason we want random effects is is because we expect the mobility scores within doctors may be correlated. There are many reasons why this could be. 

There are many reasons why this could be. For example, doctors may have specialties that mean they tend to see lung cancer patients with particular symptoms or some doctors may see more advanced cases, such that within a doctor, patients are more homogeneous than they are between doctors.

Putting this example back into matrix notation for the $n_j$ dimensional response $\mathbf{y_j}$ for doctor $j$ we would have:

$$
\overbrace{\mathbf{y_j}}^{n_j \times 1} \quad = \quad
\overbrace{\underbrace{\mathbf{X_j}}_{n_j \times 6} \quad \underbrace{\boldsymbol{\beta}}_{6 \times 1}}^{n_j \times 1} \quad + \quad
\overbrace{\underbrace{\mathbf{Z_j}}_{n_j \times 1} \quad \underbrace{\boldsymbol{u_j}}_{1 \times 1}}^{n_j \times 1} \quad + \quad
\overbrace{\boldsymbol{\varepsilon_j}}^{n_j \times 1}
$$

and by stacking observations from all groups together, since $q=1$ for the random intercept model, $qJ = (1)(407)= 407$ so we have:

Because we are only modelling random intercepts, it is special matrix in our case study that only codes which doctor a patient belongs to. If we added a random slope the number of rows in $\boldsymbol{Z}$ would remain the same but the number of columns would double. 

If we estimated it $\boldsymbol{u}$ would be a column vector similar to $\boldsymbol{\beta}$. However in classical statistics, we do not actually estimate $\boldsymbol{u}$, instead we tend to assume that 

$$\boldsymbol{u} \sim \mathcal{N}(0, \mathbf{G})$$,

Where $\mathbf{G}$ is the variance-covariance matrix of random effects. Because our example had only a random intercept $\mathbf{G}$ is a $1 \times 1$ matrix

$$
\mathbf{G} =
\begin{bmatrix}
\sigma^{2}_{int} & \sigma^{2}_{int,slope} \\
\sigma^{2}_{int,slope} & \sigma^{2}_{slope}
\end{bmatrix}
$$

Because $\mathbf{G}$ is a variance-covariance matrix, we known it should be square, symmetric and positive semi-definite. 

To simplify the computation we remove redundant effects and ensure that the resulting matrix is positive definite rather than model $\mathbf{G}$ directly, we estimate $\mathbf{\theta}$, which is a triangular Cholesky factorisation $\mathbf{G= LDL^T}$. $\mathbf{\theta})$ is not always parametrised in the same way, but you can generally thing of it as representing the random effects. It is usually designed to contain non redundant elements (unlike the variance covariance matrix) and to be parametrised in a way that yeilds more stable estimates than variances (such as taking the natural logaithm to ensure the variances are positive). Regardless of the specifics we can say that

$$\mathbf{G} = \sigma{(\mathbf{\theta})}$$

In other words, $\mathbf{G}$ is some function of $\mathbf{\theta}$. So we can get some estimate of $\mathbf{\theta}$ which we call $\mathbf{\hat{\theta}}$. Various parametrisations and constraints allow us to simplify the model for example by assuming that the random effects are independent, which would imply the true structure is 

$$
\mathbf{G} =
\begin{bmatrix}
\sigma^{2}_{int} & 0 \\
0 & \sigma^{2}_{slope}
\end{bmatrix}
$$

The final element in the model is the variance-covariance matrix of the residuals $\varepsilon$ or the variance-covariance matrix of conditional distribution of $(\mathbf{y|\beta;u}=u)$

The most common variance-covariance matrix structure is 

$$
\mathbf{R} = \boldsymbol{I}\sigma^2_{\varepsilon}
$$

Wher $\boldsymbol{I}$ is the identity matrix and $\sigma^2_{\varepsilon}$ is the residual variance. This structure assumes a homogenous residual variance for all observations and that they are conditionally independent. Other structures can be assumed. 

So the final fixed elements are $\mathbf{y}, \boldsymbol{X, Z}$ and $\boldsymbol{\varepsilon}$. The final estimated elements are $\boldsymbol{\hat{\beta},\hat{\theta}}$ and $\boldsymbol{\hat{R}}$. The final model then depends on the distribution used but is usually of the form

$$
(\mathbf{y} | \boldsymbol{\beta}; \boldsymbol{u} = u) \sim
\mathcal{N}(\boldsymbol{X\beta} + \boldsymbol{Z}u, \mathbf{R})
$$

